In [ ]:
!pip install transformers datasets accelerate -U
!pip install rouge_score
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 17.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=40f0c93205c16a3a0389bd7138696219c60afc93b85d10cbf206cac89ece826b
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.5 MB/s eta 0:00:00


In [ ]:
# --- 1. Define Model Configurations ---
MODEL_CONFIGS = [
    {
        "name": "BART-Large (CNN)",
        "model_id": "facebook/bart-large-cnn",
        "max_input_length": 1024,
        "target_max_length": 150
    },
    {
        "name": "T5-Small",
        "model_id": "t5-small",
        "max_input_length": 512,
        "target_max_length": 128
        # T5 often requires a task prefix, handled in the preprocessing function
    },
    {
        "name": "PEGASUS-XSUM",
        "model_id": "google/pegasus-xsum",
        "max_input_length": 512, # PEGASUS often uses shorter inputs
        "target_max_length": 64   # PEGASUS-XSUM targets short, 'extreme' summaries
    }
]

# Select which configuration to use
CURRENT_MODEL_CONF = MODEL_CONFIGS[0] # Change index to 1 for T5, 2 for PEGASUS
MODEL_ID = CURRENT_MODEL_CONF["model_id"]
MAX_INPUT_LENGTH = CURRENT_MODEL_CONF["max_input_length"]
TARGET_MAX_LENGTH = CURRENT_MODEL_CONF["target_max_length"]

print(f"--- Selected Model: {CURRENT_MODEL_CONF['name']} ({MODEL_ID}) ---")

--- Selected Model: BART-Large (CNN) (facebook/bart-large-cnn) ---


In [ ]:
import torch
import evaluate
import numpy as np
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq
)

# --- Configuration from Step 1 is used here ---

# 2. Load Dataset
# Using a small subset of the SAMSum dataset for a quick example
raw_datasets = load_dataset("knkarthick/samsum")

# 3. Load Tokenizer and Model
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_ID)

# T5 models require a prefix, for other models it's harmless/ignored
if "t5" in MODEL_ID:
    prefix = "summarize: "
else:
    prefix = ""

# 4. Preprocessing Function (Tokenization)
def preprocess_function(examples):
    # Apply prefix for models like T5
    inputs = [prefix + doc for doc in examples["dialogue"]]
    model_inputs = tokenizer(
        inputs,
        max_length=MAX_INPUT_LENGTH,
        truncation=True
    )

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["summary"],
            max_length=TARGET_MAX_LENGTH,
            truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply preprocessing to all splits
tokenized_datasets = raw_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=["id", "dialogue", "summary"]
)

# Use small portions for a faster demo
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["validation"].shuffle(seed=42).select(range(100))

# 5. Define Data Collator and Metrics
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=-100,
    pad_to_multiple_of=8
)

# Load ROUGE metric
metric = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    # Decode predictions and labels
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # ROUGE expects a list of references, so we use the decoded labels directly
    result = metric.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True
    )

    # Extract the median scores for a cleaner view
    # Modified: If 'v' is already a float, use it directly. Otherwise, access .mid.fmeasure.
    processed_result = {}
    for k, v in result.items():
        if isinstance(v, (float, np.floating)):
            processed_result[k] = v * 100
        elif hasattr(v, 'mid') and hasattr(v.mid, 'fmeasure'):
            processed_result[k] = v.mid.fmeasure * 100
        else:
            # Fallback for unexpected types, or if the structure has changed in a different way
            print(f"Warning: Unexpected ROUGE score type for '{k}': {type(v)}. Attempting direct conversion.")
            try:
                processed_result[k] = float(v) * 100
            except (TypeError, ValueError):
                print(f"Error: Could not convert ROUGE score for '{k}' with value '{v}' to float.")
                processed_result[k] = 0.0 # Assign a default or handle as appropriate
    result = processed_result

    # Add generation length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

# 6. Define Training Arguments and Trainer
training_args = Seq2SeqTrainingArguments(
    output_dir=f"./summarization_finetuning/{CURRENT_MODEL_CONF['name']}",
    num_train_epochs=3,                     # **Parameter Change: Number of Epochs**
    per_device_train_batch_size=4,          # **Parameter Change: Training Batch Size**
    per_device_eval_batch_size=4,           # **Parameter Change: Evaluation Batch Size**
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_strategy="epoch",                  # Save checkpoint after each epoch
    eval_strategy="epoch",            # Evaluate after each epoch
    learning_rate=2e-5,                     # **Parameter Change: Learning Rate**
    fp16=torch.cuda.is_available(),         # Use mixed precision if a GPU is available
    predict_with_generate=True,             # Essential for generation tasks
    generation_max_length=TARGET_MAX_LENGTH # Use the model-specific max length
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# 7. Start Training
print("\n--- Starting Training ---")
trainer.train()

print("\n--- Training Complete ---")
print(f"Final Evaluation Metrics:\n{trainer.evaluate()}")

Map:   0%|          | 0/14731 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

/tmp/ipython-input-731188975.py:131: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(



--- Starting Training ---


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.368300,1.584098,40.735300,19.869300,31.201700,31.189900,60.050000
2,1.246300,1.558801,40.899600,20.398200,30.903600,30.863900,60.070000
3,0.806600,1.630622,42.571300,20.078400,31.812600,31.767800,60.760000


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(



--- Training Complete ---


Final Evaluation Metrics:
{'eval_loss': 1.630622386932373, 'eval_rouge1': 42.5713, 'eval_rouge2': 20.0784, 'eval_rougeL': 31.8126, 'eval_rougeLsum': 31.7678, 'eval_gen_len': 60.76, 'eval_runtime': 46.6072, 'eval_samples_per_second': 2.146, 'eval_steps_per_second': 0.536, 'epoch': 3.0}
